<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/japanesenlp/huggingfacenlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title bert model tutorial
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2023-02-13 00:37:12--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.66MB/s    in 3.2s    

2023-02-13 00:37:16 (2.66 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [ ]:
!tar zxvf ldcc-20140209.tar.gz

In [3]:
# create one csv file
import os
import pandas as pd

def get_title_list(path):
    """記事タイトル取得関数"""
    title_list = []
    filenames = os.listdir(path) #ファイル名称一覧取得
    for filename in filenames:
        # 1記事ずつファイルの読み込み
        with open(path+filename, encoding="utf_8_sig") as f:
            title = f.readlines()[2].strip() #各記事テキストの改行2番目に記事タイトルが記載してある
            title_list.append(title)
    return title_list

In [4]:
df = pd.DataFrame(columns=['label', 'sentence']) #空データフレーム

#独女通信(ラベル0)
title_list = get_title_list('/content/text/dokujo-tsushin/')
for title in title_list:
    df = df.append({'label':0 , 'sentence':title}, ignore_index=True) #ignore_indexで合体後のindexを連番に

#ITライフハック(ラベル1)
title_list = get_title_list('/content/text/it-life-hack/')
for title in title_list:
    df = df.append({'label':1 , 'sentence':title}, ignore_index=True)

#MOVIE ENTER(ラベル2)
title_list = get_title_list('/content/text/movie-enter/')
for title in title_list:
    df = df.append({'label':2 , 'sentence':title}, ignore_index=True)

# 全データの順番をシャッフル(+index振り直し)
df = df.sample(frac=1 ,random_state=0).reset_index(drop=True)

#一応csvとしても保存しておく
df.to_csv('livedoor_sentence.csv', sep=',', index=False, encoding='utf_8_sig')

In [5]:
df.head()

,label,sentence
0,1,サムスンがテーブル型PCを投入！Surface 2.0を搭載するPCの魅力
1,2,北乃きい、東日本大震災が起きて感じたこと
2,2,深田恭子「女性の私が見ても、ときめいた」
3,2,ミスチルが書き下ろした主題歌とともに、初恋の出会いを綴る
4,2,【DVDエンター！】水樹奈々が声優出演、大ヒットドラマの美人チアリーダーが超美形俳優と恋に


In [ ]:
# check label condition
import collections
print(collections.Counter(df['label']))

In [ ]:
!pip install transformers[ja]

In [ ]:
from transformers import BertJapaneseTokenizer
"""
・BertJapaneseTokenizerは日本語用のBERTトークナイザ
・from_pretrainedで指定されたPytorchモデルの事前学習を実行する
・"cl-tohoku/bert-base-japanese-v2"は東北大学の日本語事前学習用BERTモデル ※下記補足参照
・do_subword_tokenizeは、サブワードのトークン化をするかどうか
・mecab_kwargsでMeCabユーザー辞書やNeoLogd等の指定も可能
　※"mecab_dic": Noneでデフォルト辞書(UniDic)をOFFにする必要あり
"""
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-v2", 
    #do_subword_tokenize=False,
    #mecab_kwargs={"mecab_dic": None, "mecab_option": "-d 'C:\mecab-unidic-neologd'"
)

In [9]:
#適当なキーワードでトークナイズしてみる
text = "楽しくリズム感覚が身につく"

#tokenizer.encodeでテキストをトークンIDに,return_tensors='pt'でPytorch型のテンソル型に変換
ids = tokenizer.encode(text, return_tensors='pt')[0]
wakati = tokenizer.convert_ids_to_tokens(ids) #どのようにトークナイズされたか分かち書きで確認
print(ids)
print(wakati)

tensor([    2, 32589, 17651, 16947,   862,  5128,   893, 12953,     3])
['[CLS]', '楽しく', 'リズム', '感覚', 'が', '身', 'に', 'つく', '[SEP]']


In [10]:
# create bert classification
"""
・BertForSequenceClassificationはBEETの最終層をクラス分類に変えたもの
・事前学習はトークナイザと同じものを指定する
・num_labelsで分類数を指定する
"""

import torch
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-v2",
    num_labels = 3
)

#学習モードに設定
model.train()

#使用デバイス設定(CPU,GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device) #modelをGPUに転送

#オプティマイザの設定
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [11]:
#乱数のseedを全固定する
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(0)

In [12]:
# create training dataset
from torch.utils.data import TensorDataset, random_split, DataLoader, SequentialSampler, RandomSampler
from sklearn.model_selection import train_test_split

"""
・encodingで分類対象の文をトークナイザ
・input_idsはトークンID
・attention_mask はパディング用に埋め込み文字化どうかの判断用
・train_labels は分類ラベル
"""
encoding = tokenizer(df['sentence'].tolist(), return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
train_labels = torch.tensor(df['label'].tolist())

#データセット作成
dataset = TensorDataset(input_ids, attention_mask, train_labels)

#学習とテストの割合　※ここでは9割学習
train_size = int(0.9*len(dataset))
val_size= len(dataset) - train_size
train_dataset, val_dataset= random_split(dataset, [train_size, val_size])

print("学習データ数: {}" .format(train_size))
print("テストデータ数: {}" .format(val_size))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


学習データ数: 2351
テストデータ数: 262


In [13]:
# for check in order to confirm the effect of tranining
import pandas as pd
import numpy as np

for i in range(val_dataset.__len__()):
    #別の方法あるかもだが、文字置換でスペシャルトークン(CLSとか)を消している
    sentence = tokenizer.decode(val_dataset.__getitem__(i)[0].detach().numpy().tolist()).replace('[CLS]', '').replace('[PAD]', '').replace('[SEP]', '').replace(' ', '')
    df_val_sentence_kari = pd.DataFrame(sentence.split(), columns={"sentence"})
        
    if i==0:
        df_val_sentence = df_val_sentence_kari.copy()
    else:
        df_val_sentence = pd.concat([df_val_sentence_kari, df_val_sentence], axis=0)

df_val_sentence.head(2) #お試しで2行表示

,sentence
0,【オトナ女子映画部】“女”に生まれ、“母”になる『愛する人』
0,「Dropbox」のUbuntu版を試すLinuxでも同じ使いこなし!【デジ通】


In [14]:
# create dataloader
batch_size = 8

train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),
    batch_size = batch_size
)

validation_dataloader = DataLoader(
    val_dataset,
#     sampler = RandomSampler(val_dataset), #先ほど元sentenceを保存した為シャッフルしない
    batch_size = 1 #テストなので1にした
)

In [15]:
def train(model):
    """学習ループ用関数"""
    model.train()
    train_loss = 0
    
    for batch in train_dataloader:
        b_input_ids = batch[0].to(device) #トークンID
        b_input_mask = batch[1].to(device) #埋め込み文字判断
        b_labels = batch[2].to(device) #正解ラベル
        optimizer.zero_grad()
        
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()    
    return train_loss

max_epoch = 10 #Epoch数は適当に
train_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    train_loss_.append(train_)
    
    print(train_)

107.0989949060604
29.874469569534995
11.498682274250314
5.23470992653165
3.985955436146469
2.500752098232624
1.0637855646127719
0.7444356073247036
1.5191888657136587
0.702425707691873


In [16]:
# inference
model.eval() #検証モード
preds_list = []
b_labels_list = []

for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device) #ラベルはモデルへ入力しないが正解確認用
    
    with torch.no_grad():
        preds = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        preds_list.append(preds) #予測ラベル
        b_labels_list.append(b_labels) #正解ラベル

#データフレームで結果を可視化する
for i in range(len(preds_list)):
    #preds_list[i][0]で一番確率が高いものを予測ラベルと判断(よくある分類のパターンと同じ)
    df_pred = pd.DataFrame(np.argmax(preds_list[i][0].cpu().numpy(), axis=1), columns={"pred_label"})
    df_label = pd.DataFrame(b_labels_list[i].cpu().numpy(), columns={"true_label"})
    df_result_kari = pd.concat([df_pred, df_label], axis=1)
    
    if i==0:
        df_result = df_result_kari.copy()
    else:
        df_result = pd.concat([df_result_kari, df_result], axis=0)

df_result = pd.concat([df_val_sentence, df_result], axis=1) #元タイトル(2-3で準備)と合体

df_result.head(10) #お試しで2行表示

,sentence,pred_label,true_label
0,【オトナ女子映画部】“女”に生まれ、“母”になる『愛する人』,0,0
0,「Dropbox」のUbuntu版を試すLinuxでも同じ使いこなし!【デジ通】,1,1
0,何これ欲しい!のキーボード登場!エレコムのレーザー投影式キーボード,1,1
0,GIGABYTEマーケティング部門副部長が語る!アキバで見たシニア世代に感動,1,1
0,懐かしのヒーローが続々!名アニメーターが描くヒーローイラストが話題に,2,1
0,意外と使える海外ドラマ『弁護士イーライ』のビジネス術「情報に含まれる“心”を読み取れ!」,2,2
0,伝説が、壮絶に、終わる!『ダークナイトライジング』最後の予告編が解禁に,2,2
0,日本のセンスが認められた!『メリダとおそろしの森』のポスターに注目,2,2
0,【編集部的映画批評】映画『るろうに剣心』は原作ファンに斬られるのか?,2,2
0,出演者は130人超、前夜祭も開催!真夏のダンマス4は超熱い!,2,1


In [ ]:
#@title More simple

In [18]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import os
import random
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

import transformers
from transformers import (
    AutoModel, AutoTokenizer, EvalPrediction, Trainer,
    TrainingArguments, AutoModelForSequenceClassification,
    EarlyStoppingCallback,
)
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import BertPreTrainedModel, BertModel

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from torchinfo import summary

#乱数固定
def seed_everything(seed: int):
    """seed固定"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 7
seed_everything(SEED)

In [20]:
def get_title_list(path):
    """記事タイトル取得関数"""
    title_list = []
    filenames = os.listdir(path) #ファイル名称一覧取得
    for filename in filenames:
        # 1記事ずつファイルの読み込み
        with open(path+filename, encoding="utf_8_sig") as f:
            title = f.readlines()[2].strip()
            title_list.append(title)
    return title_list

#コーパスをDLして格納したディレクトリ
DATA_DIR = "/content/text"

#空データフレームを用意
df = pd.DataFrame(columns=['label', 'sentence'])

#用意した空データフレームにappendしていく
#独女通信(ラベル0)
title_list = get_title_list(DATA_DIR + '/dokujo-tsushin/')
for title in title_list:
    df = df.append({'label':0 , 'sentence':title}, ignore_index=True)
#ITライフハック(ラベル1)
title_list = get_title_list(DATA_DIR + '/it-life-hack/')
for title in title_list:
    df = df.append({'label':1 , 'sentence':title}, ignore_index=True)
#MOVIE ENTER(ラベル2)
title_list = get_title_list(DATA_DIR + '/movie-enter/')
for title in title_list:
    df = df.append({'label':2 , 'sentence':title}, ignore_index=True)

# 全データの順番をシャッフル(+index振り直し)
df = df.sample(frac=1 ,random_state=0).reset_index(drop=True)

#お試しで2行表示
df.head(2)

,label,sentence
0,1,サムスンがテーブル型PCを投入！Surface 2.0を搭載するPCの魅力
1,2,北乃きい、東日本大震災が起きて感じたこと


In [21]:
# train, test, val split
#最初の6割をtrain、次の2割をvalid、最後の2割をtestでデータフレームを分割
train, val, test = np.split(df, [int(len(df) * .6), int(len(df) * .8)])

print(train.shape)
print(val.shape)
print(test.shape)
#お試しで1行ずつ表示
display(train.head(1))
display(val.head(1))
display(test.head(1))

(1567, 2)
(523, 2)
(523, 2)


,label,sentence
0,1,サムスンがテーブル型PCを投入！Surface 2.0を搭載するPCの魅力


,label,sentence
1567,0,「愛はないけど嫌いじゃない」離婚に踏み切らない女たちの本音


,label,sentence
2090,0,「愛している」と言っていますか？


In [22]:
MODEL_NAME = "cl-tohoku/bert-base-japanese-v2"

#東北大Wikipediaモデルの2020年8月実行＋BERT-Baseモデル版を指定
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = MODEL_NAME)

ids = tokenizer.encode(train['sentence'][0])
wakati = tokenizer.convert_ids_to_tokens(ids)
print(train['sentence'][0])
print(ids)
print(wakati)

サムスンがテーブル型PCを投入！Surface 2.0を搭載するPCの魅力
[2, 14776, 27882, 862, 18264, 1752, 13505, 932, 14353, 15, 30297, 6249, 15757, 32, 28, 30, 932, 11770, 11137, 13505, 896, 17098, 3]
['[CLS]', 'サム', '##スン', 'が', 'テーブル', '型', 'PC', 'を', '投入', '!', 'Sur', '##f', '##ace', '2', '.', '0', 'を', '搭載', 'する', 'PC', 'の', '魅力', '[SEP]']


In [24]:
class BertClassificationModel(BertPreTrainedModel):
  def __init__(self, config):
    super().__init__(config)
    self.num_labels = config.num_labels
    self.config = config
    self.bert = BertModel(config)
    self.dropout = nn.Dropout(0.5)
    self.linear1 = nn.Linear(768, 100)
    self.linear2 = nn.Linear(100, self.num_labels)
  def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, output_attentions=None, output_hidden_states=None):
    outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # change linear conversion and finally decompose 3 dimension
    outputs = self.dropout(outputs.last_hidden_state[:, 0, :])
    outputs = self.linear1(outputs)
    outputs = self.linear2(outputs)
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(outputs.view(-1, self.num_labels), labels.view(-1))

    #SequenceClassifierOutputは文章分類用のデータクラス
    return  SequenceClassifierOutput(
            loss=loss,
            logits=outputs,
            hidden_states=output_hidden_states,
            attentions=output_attentions,
        )

In [ ]:
class MyBertClassyModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        self.bert = BertModel(config)
        #以下は線形部分。768から一気に3次元(分類数=3)にせずに100次元を追加
        self.dropout = nn.Dropout(0.5)
        self.linear1 = nn.Linear(768, 100) # 768⇒100
        self.linear2 = nn.Linear(100, 3) # 100⇒3
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None\
        , labels=None, output_attentions=None, output_hidden_states=None):
        #BERTで768次元のベクトルを出力
        outputs = self.bert(
            input_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        #線形変換し、最終的には3次元まで次元削減する
        outputs = self.dropout(outputs.last_hidden_state[:, 0, :])
        outputs = self.linear1(outputs) # 768⇒100
        outputs = self.linear2(outputs) # 100⇒3
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(outputs.view(-1, self.num_labels), labels.view(-1))
        
        #SequenceClassifierOutputは文章分類用のデータクラス
        return  SequenceClassifierOutput(
            loss=loss,
            logits=outputs,
            hidden_states=output_hidden_states,
            attentions=output_attentions,
        )

In [ ]:
# モデルのインスタンス化
model = BertClassificationModel.from_pretrained(MODEL_NAME, num_labels=3)

#簡単にやりたいならわざわざ自分でclass定義せずに以下だけでOK
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels = 3)

# torchinfoで中身を確認
summary(model, depth=4)

In [ ]:
#まずは定義したmodelの全層の購買を更新させなくする ※print(name)で層の名称を確認可能
for name, param in model.named_parameters():
    # print(name)
    param.requires_grad = False

#BERTの最終層「self.bertのencoder層ラスト(-1)」だけ勾配更新するようにする
for name, param in model.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

#さらに、自分で定義したモデルの「全結合層(self.linear1/self.linear2)」だけ勾配更新するようにする
for name, param in model.linear1.named_parameters():
    param.requires_grad = True
for name, param in model.linear2.named_parameters():
    param.requires_grad = True

#最後に勾配更新対象の層を確認する
for name, param in model.named_parameters():
    if param.requires_grad : print(name)

In [30]:
# create dataset
text_lengths_list = [len(tokenizer.encode(text)) for text in df["sentence"].to_list()]

train_X = tokenizer(train["sentence"].tolist(), return_tensors='pt', padding="max_length", max_length=max(text_lengths_list), truncation=True)
valid_X = tokenizer(val["sentence"].tolist(), return_tensors='pt', padding="max_length", max_length=max(text_lengths_list), truncation=True)
test_X = tokenizer(test["sentence"].tolist(), return_tensors='pt', padding="max_length", max_length=max(text_lengths_list), truncation=True)

print(f"最大トークン数は{max(text_lengths_list)}です")
print(train_X.keys())
print(len(train_X['input_ids']))
print(len(valid_X['input_ids']))
print(len(test_X['input_ids']))

最大トークン数は47です
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
1567
523
523


In [31]:
class MyDataset(Dataset):
  def __init__(self, encodings, labels=None):
    self.encodings = encodings
    self.labels = labels

  def __len__(self):
    return len(self.encodings['input_ids'])

  def __getitem__(self, index):
    input = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
    if self.labels is not None:
        input["label"] = torch.tensor(self.labels[index])

    return input

#train/valid/testのデータセットをそれぞれ作成する ※testは当然label無し
train_ds = MyDataset(train_X, train["label"].tolist())
valid_ds = MyDataset(valid_X, val["label"].tolist())
test_ds = MyDataset(test_X)

#お試し確認
print(train_ds[0].keys())
print(f"学習データ数は{len(train_ds)}です")
print(tokenizer.decode(train_ds[0]["input_ids"])) #デコードで戻してみる

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])
学習データ数は1567です
[CLS] サムスン が テーブル 型 PC を 投入! Surface 2. 0 を 搭載 する PC の 魅力 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [32]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    #2値分類ならaverage='binary'とする
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [33]:
# Training Arguments
train_args = TrainingArguments(
    output_dir                  = "./out", #log出力場所
    overwrite_output_dir        = True, #logを上書きするか
    load_best_model_at_end      = True, #EarlyStoppingを使用するならTrue
    metric_for_best_model       = "f1", #EarlyStoppingの判断基準。7-1. compute_metricsのものを指定
    save_total_limit            = 1, #output_dirに残すチェックポイントの数
    save_strategy               = "epoch", #いつ保存するか？
    evaluation_strategy         = "epoch", #いつ評価するか？
    logging_strategy            = "epoch", #いつLOGに残すか？
    label_names                 = ['labels'], #分類ラベルのkey名称(デフォルトはlabelsなので注意)
    lr_scheduler_type           = "linear", #学習率の減衰設定(デフォルトlinearなので設定不要)
    learning_rate               = 5e-5, #学習率(デフォルトは5e-5)
    num_train_epochs            = 3, #epoch数
    per_device_train_batch_size = 16, #学習のバッチサイズ
    per_device_eval_batch_size  = 12, #バリデーション/テストのバッチサイズ
    seed                        = SEED, #seed
)

In [34]:
# early stopping
MyCallback = EarlyStoppingCallback(early_stopping_patience=3)

In [ ]:
trainer = Trainer(
    model=model, #モデル
    args=train_args, #TrainingArguments
    tokenizer=tokenizer, #tokenizer
    train_dataset=train_ds, #学習データセット
    eval_dataset=valid_ds, #validデータセット
    compute_metrics = compute_metrics, #compute_metrics
    callbacks=[MyCallback] #callback
)

trainer.train()

In [37]:
test_preds = trainer.predict(test_ds)

***** Running Prediction *****
  Num examples = 523
  Batch size = 12


In [ ]:
test['pred'] = np.argmax(test_preds.predictions, axis=1)
test.head(2)

In [47]:
precision, recall, f1_score, _ = precision_recall_fscore_support(test['label'].values.astype('int64'), test['pred'].values, average=None)

In [49]:
print('正答率（Accuracy） = {:.3f}%'.format(100 * accuracy_score(test['label'].values.astype('int64'), test['pred'].values))) # 正答率を表示
print('適合率（Precision） = {:.3f}%'.format(100 * precision[0])) # 適合率を表示
print('再現率（Recall） = {:.3f}%'.format(100 * recall[0])) # 再現率を表示
print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[0])) #F1値を表示

#dfもお試し表示
display(test.head(5))

正答率（Accuracy） = 95.220%
適合率（Precision） = 93.529%
再現率（Recall） = 94.083%
F1値（F1-score） = 93.805%


,label,sentence,pred
2090,0,「愛している」と言っていますか？,0
2091,2,“世界が敬愛する映画人”スコセッシ監督の最高傑作、技術賞を総ナメ,2
2092,0,あなたは何様ですか？,0
2093,1,地方の元気が日本の元気に！「ニコニコ町会議」佐賀県唐津市呼子町に1万6千人が来場,1
2094,2,せっかくの3D恐怖動画なのに、はしゃぎ過ぎの満島ひかり,2
